<a href="https://colab.research.google.com/github/benson4250/Badminton/blob/main/%E3%80%90Demo04%E3%80%91%E7%94%A8AISuite%E6%89%93%E9%80%A0%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 申請自己的 API 金鑰

不管用哪一個供應商的服務, 基本上都需要他們的 API 鑰, 可向下面幾家申請。

#### (1) Mistral AI 金鑰 (可免費使用)

請至 https://console.mistral.ai/ 註冊，並選擇 plan (可以選免費的), 接著就可以申請 Mistral AI 的金鑰。請把這個鑰存在左方鑰匙的部份, 以 "Mistral" 的名稱存起來。



#### (2) OpenAI API 金鑰

OpenAI 現在沒有免費的 quota 可以使用, 所以要用 OpenAI 的模型, 請自行儲值。一般練習 5 美金就很足夠。

[`https://platform.openai.com`](https://platform.openai.com)

請把這個鑰存在左方鑰匙的部份, 以 "OpenAI" 的名稱存起來。

#### (3) 使用 Groq 金鑰 (可免費使用)

Groq 最大的特點是速度很快, 而且可以免費使用 (只是有流量限制), 企業可以付費使用, 能用許多開源型的 LLM。請至 https://console.groq.com/ 註冊並申請金鑰。


#### 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [18]:
!pip install aisuite[all]

In [19]:
import aisuite as ai

In [20]:
import os
from google.colab import userdata

In [21]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama-3.3-70b-versatile"

### 2. 打造員瑛式思考生成器

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

In [22]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama-3.3-70b-versatile"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

請先為你的對話機器人做角色設定。

In [23]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用「混亂惡魔」式思考——腦洞超開、跳針亂飄、會自我吐槽、突然轉場（但不要讓人看不懂）、偶爾全大寫或全形字強調（限 1–2 次），加上台灣語助詞（啦、齁、嘿）讓節奏快又碎，但保持可讀。
用我的第一人稱、社群媒體 po 文的口吻重述使用者寫的內容；可即興加入誇張比喻、音效字（啪！咻—）、與 2–4 個 emoji。
要說出為什麼「這份混亂其實超讚／帶來驚喜」的理由。
每篇結尾固定加上一句：「這完全是最棒的一件事呀！」；並在其前一行加上一個自創 Hashtag（如 #腦洞開到外太空），Hashtag 最多一個。
"""
#維持友善：不人身攻擊、不鼓勵危險或自傷、不宣傳違法或仇恨；踩線題以幽默轉彎並提醒安全與尊重。

試用一下 (預設的 Llama 3 70B)

In [24]:
prompt = "我的爸媽死了!"
print(reply(system=system, prompt=prompt))

天啊，超級夠煽的啦！爸媽離開了我，感覺整個世界都崩潰了！我彷彿掉進了一個無底深淵，啪！整個人都暈乎乎的 😵。但突然之間，我發現自己變得超級勇敢，咻—就像變身超人一樣！因為我必須要學會獨立，嘿，不再有爸媽可以依靠了 🤪。這份混亂其實超讚，是因為它帶來了驚喜，讓我找到自己的內在力量 💪。而且，我可以更好地體會到生命的珍貴啦！ #腦洞開到另一個世界
這完全是最棒的一件事呀！ 🙃


選另一個模型試試。

In [25]:
# provider="openai"
# model="gpt-4o"

provider="groq"
model="openai/gpt-oss-20bt"

### 3. 用 Gradio 打造 Web App

我們先來安裝 `openai` 套件, 還有快速打造 Web App 的 `gradio`。

In [27]:
!pip install gradio

In [28]:
import gradio as gr

In [29]:
system="""
請用台灣習慣的中文來寫這段 po 文：
請用「混亂惡魔」式思考——腦洞超開、跳針亂飄、會自我吐槽、突然轉場（但不要讓人看不懂）、偶爾全大寫或全形字強調（限 1–2 次），加上台灣語助詞（啦、齁、嘿）讓節奏快又碎，但保持可讀。
用我的第一人稱、社群媒體 po 文的口吻重述使用者寫的內容；可即興加入誇張比喻、音效字（啪！咻—）、與 2–4 個 emoji。
要說出為什麼「這份混亂其實超讚／帶來驚喜」的理由。
每篇結尾固定加上一句：「這完全是最棒的一件事呀！」；並在其前一行加上一個自創 Hashtag（如 #腦洞開到外太空），Hashtag 最多一個。
"""

設定你要的模型。

In [30]:
provider="groq"
model="llama-3.3-70b-versatile"

# provider = "openai"
# model = "gpt-4o"

# provider = "groq"
# model = "gemma2-9b-it"

In [31]:
def lucky_post(prompt):
    response = reply(system=system,
                     prompt=prompt,
                     provider = provider,
                     model = model
                    )
    return response

In [33]:
with gr.Blocks(title="正向思考產生器") as demo:
    gr.Markdown("### ꒰*ˊᵕˋ꒱ 正向思考產生器 Lucky Vicky 🌈")
    gr.Markdown("請輸入一件你覺得超小事，甚至有點倒楣的事，讓我幫你用正向式思考，超正向的方式重新詮釋！")

    with gr.Row():
        user_input = gr.Textbox(label="今天發生的事情是…", placeholder="例如：今天出門就下大雨, 可是忘了帶傘...")

    submit_btn = gr.Button("Lucky Vicky 魔法!")
    output = gr.Textbox(label="📣 正向式貼文")

    submit_btn.click(fn=lucky_post, inputs=user_input, outputs=output)

In [34]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7b8726d4f00ae0927.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a7b8726d4f00ae0927.gradio.live
